In [1]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import json

In [2]:
# Load the pre-trained emotion analysis pipeline
emotion_pipeline = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

Device set to use mps:0


In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
print("Model Token Limit:", tokenizer.model_max_length)

Model Token Limit: 512


In [3]:
with open("final_campaign_data.txt", "r") as file:
    data = json.load(file)

In [53]:
# Convert the JSON data to a Pandas DataFrame
campaigns = pd.DataFrame(data)

In [52]:
# for index, row in campaigns.iterrows():
#     description = " ".join(row["Description"].split())
#     tokens = tokenizer(description, return_tensors="pt", truncation=False)["input_ids"][0]
#     chunks = [tokens[i: i + 20] for i in range(0, len(tokens), 10)]
#     print(chunks[0])
#     print(chunks[1])
#     break

tensor([    0, 43309,  3869,  2981, 21721,   571, 10564,    16, 13916,    42,
        11137,     4,  2497,   183,    82,     9,   476,    25,     5,  1270])
tensor([11137,     4,  2497,   183,    82,     9,   476,    25,     5,  1270,
          161,   112,  1404,     7,    65,   153,  1932,    38,   524,  5162])


In [72]:

# def classify_emotion(description):
#     try:
#         result = emotion_pipeline(description, emotionally_charged_threshold)
#         dominant_emotion = max(result, key=lambda x: x['score'])
#         is_emotionally_charged = (
#             dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral"
#         )
#         return dominant_emotion['label'], dominant_emotion['score'], is_emotionally_charged
#     except Exception as e:
#         return "Error", 0.0, False

TOKEN_LIMIT = tokenizer.model_max_length

# Function to classify emotions with a sliding window approach for long texts
def classify_emotion_sliding_window(description, emotionally_charged_threshold):
    try:
        description = " ".join(description.split())
        # Tokenize the text to check length
        tokens = tokenizer(description, return_tensors="pt", truncation=False)["input_ids"][0]

        if len(tokens) <= TOKEN_LIMIT:
            # If text fits within the token limit, classify normally
            result = emotion_pipeline(description)
            dominant_emotion = max(result, key=lambda x: x['score'])
            is_emotionally_charged = dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral"
            return dominant_emotion['label'], dominant_emotion['score'], is_emotionally_charged

        # If text exceeds the token limit, apply sliding window approach
        stride = TOKEN_LIMIT // 2  # Overlapping stride of 50%
        chunks = [tokens[i: i + TOKEN_LIMIT] for i in range(0, len(tokens), stride)]

        highest_emotion = "neutral"
        highest_score = 0.0
        is_emotionally_charged = False

        for chunk in chunks:
            decoded_text = tokenizer.decode(chunk, skip_special_tokens=True)
            result = emotion_pipeline(decoded_text)
            dominant_emotion = max(result, key=lambda x: x['score'])

            # Check if this chunk has the highest emotion score so far
            if dominant_emotion['score'] > highest_score and dominant_emotion['label'] != "neutral":
                highest_emotion = dominant_emotion['label']
                highest_score = dominant_emotion['score']
            # If any chunk has a score higher than the threshold, mark as emotionally charged
            if dominant_emotion['score'] >= emotionally_charged_threshold and dominant_emotion['label'] != "neutral":
                is_emotionally_charged = True

        return highest_emotion, highest_score, is_emotionally_charged

    except Exception as e:
        return "Error", 0.0, False


In [73]:
len(campaigns)

21993

In [75]:
import warnings
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm

tqdm.pandas()

emotionally_charged_threshold = 0.70

# Apply the function with progress tracking, passing the threshold as an argument
campaigns["DominantEmotion"], campaigns["EmotionScore"], campaigns["EmotionallyCharged"] = zip(
    *campaigns["Description"].progress_apply(lambda desc: classify_emotion_sliding_window(desc, emotionally_charged_threshold))
)


# campaigns[["DominantEmotion", "EmotionScore", "EmotionallyCharged"]] = campaigns["Description"].apply(
#     lambda desc: pd.Series(classify_emotion_sliding_window(desc, emotionally_charged_threshold))
# )
# start_time = time.time()

# # Perform emotion analysis on all campaign title
# campaigns1 = campaigns.head(20)
# campaigns1[["DominantEmotion", "EmotionScore", "EmotionallyCharged"]] = campaigns1["Description"].apply(
#     lambda desc: pd.Series(classify_emotion_sliding_window(desc, emotionally_charged_threshold))
# )
# end_time = time.time()
# execution_time = end_time - start_time  # Compute elapsed time
# print(f"Execution Time: {execution_time:.6f} seconds")

  0%|          | 0/21993 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|██████████| 21993/21993 [1:44:46<00:00,  3.50it/s]  


In [78]:
campaigns.to_json('campaigns_with_emotionally_charged_data.json') 